In [1]:
#importing important libraries
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Basic Scrapping

In [ ]:
url = 'https://webscraper.io/test-sites/e-commerce/allinone/computers' #storing the url
page = requests.get(url) #getting the access of website
soup = BeautifulSoup(page.text,'lxml') # getting the html format
soup

In [ ]:
print(soup.header.p.string) #to get the content of any tag
print(soup.header.a.attrs) #to get all the attribute as a dictionary

#to finds the first occurance
print(soup.find('h2'))
print(soup.find('div', class_ = 'navbar-header'))
print(soup.find('div',{'class': 'navbar-brand' }))

#to find all the occurances
print(soup.find_all('a', class_ = 'title'))
soup.find_all(['p','a'])

print(soup.find_all(id = True)) #to find by id
print(soup.find_all(string = 'Asus')) #to find by string exact match
print(soup.find_all(string = re.compile('Asus'))) # to find similar match
print(soup.find_all('p',class_ = re.compile('pull'),limit = 2)) #to limit scrapping
print(soup.find_all(string = [re.compile('Asus'),re.compile('White')])) #to find multiple items


In [ ]:
# make a dataframe
product_name = soup.find_all('a',class_ = 'title')
price = soup.find_all('h4',class_ = 'pull-right price')
reviews = soup.find_all('p', class_ = 'pull-right')
description = soup.find_all('p', class_ = 'description')

product_list = price_list = review_list = description_list = [] #making empty list

for i,j,k,l in zip(product_name, price, reviews, description): #looping over all the tags
    name = i.text #extracting the text
    product_list.append(name) # storing in the list
    
    p = j.text
    price_list.append(p)
    
    r = k.text
    review_list.append(r)
    
    d = l.text
    description_list.append(d)
    
table = pd.DataFrame({'Product':product_list,'Price':price_list,'Description':description_list,'Review':review_list})
table #final dataframe

# Table Scrapping

In [ ]:
url = 'https://www.nfl.com/standings/league/2022/REG'
page = requests.get(url)
soup = BeautifulSoup(page.text,'lxml')
nested = soup.find('table', {'summary': 'Standings - Detailed View'}) # getting only the table html tag
nested

In [ ]:
column_name = []
for i in nested.find_all('th'):
    name = i.text.strip() 
    column_name.append(name) # getting the header of the table
column_name

In [29]:
nfl = pd.DataFrame(columns = column_name)
nfl

,NFL Team,W,L,T,PCT,PF,PA,Net Pts,Home,Road,Div,Pct,Conf,Pct,Non-Conf,Strk,Last 5


In [ ]:
# getting the data of all the rows
for i in nested.find_all('tr')[1:]:
    first_data = i.find('td').find('div', class_ = 'd3-o-club-fullname').text.strip() # getting all the team name
    row_data = i.find_all('td')[1:]
    data = [j.text.strip() for j in row_data] #getting the data
    data.insert(0,first_data)
    length = len(nfl)
    nfl.loc[length] = data
nfl

# Airbnb Scraping

In [ ]:
#searching for hotels in goa
url = 'https://www.airbnb.co.in/s/Goa/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&query=Goa&place_id=ChIJQbc2YxC6vzsRkkDzYv-H-Oo&date_picker_type=calendar&checkin=2022-12-24&checkout=2023-01-02&flexible_date_search_filter_type=0&source=structured_search_input_header&search_type=autocomplete_click'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
soup

In [ ]:
#making the dataframe
air_df = pd.DataFrame({'link':[''],'title':[''],'name':[''],'detail':[''],'rating':[''],'price':['']})

In [ ]:
#extracting from multiple pages
while True:
    posting = soup.find_all('div', class_= 'cy5jw6o dir dir-ltr') # storing all the posting
    for post in posting:
        try:
            link = post.find('a', class_ = 'bn2bl2p dir dir-ltr').get('href') #getting the link
            full_link = 'https://www.airbnb.co.in' + link # appending the full link
            title = post.find('div', class_ = 't1jojoys dir dir-ltr').text
            name = post.find('span', class_ = 't6mzqp7 dir dir-ltr').text
            detail = post.find_all('div', class_ = 'f15liw5s s1cjsi4j dir dir-ltr')[1].text
            rating = post.find('span', class_ = 'r1dxllyb dir dir-ltr').text
            try: 
                price = post.find('span', class_ = '_tyxjp1').text # two price classes are there so try which one is present
            except:
                price = post.find('span', class_ = '_1y74zjx').text
            #finally appending into the dataframe    
            air_df = air_df.append({'link':full_link,'title':title,'name':name,'detail':detail,'rating':rating,'price':price}, ignore_index = True)
        except: # if any tag doesn't have the value then just ignore that posting and pass
            pass
    next_page = soup.find('a',{'aria-label':'Next'}).get('href') #next page link
    full_link = 'https://www.airbnb.co.in' + next_page
    page = requests.get(full_link)
    soup = BeautifulSoup(page.text, 'lxml') # getting the next page html tags   
            